<a href="https://colab.research.google.com/github/Guillermo-rv/ML/blob/main/Data_Acquisition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Session 01

In [ ]:
import pandas as pd

## Exercise 1

In [ ]:
# Reading the txt with columns names
my_columns = pd.read_csv("columns_Top_50_US_Tech_Companies_2022-2023.txt", header=None)

# Reading the csv with the data
df_data = pd.read_csv("data_Top_50_US_Tech_Companies_2022-2023.csv", sep=";", header=None, names=my_columns[0]) # my_columns[0].tolist()
# df_data.columns = my_columns[0].tolist() # Way number 2

# Showing a specific value of a row
print(df_data['Annual Revenue 2022-2023 (USD in Billions)'][0]) # .loc or .iloc also work

# Replacing , to . (decimals in python by default)
list_cols_tochange = [df_data.columns[5], df_data.columns[6], df_data.columns[8]]
df_data[list_cols_tochange] = df_data[list_cols_tochange].replace(",", ".", regex=True)
# df_data['Annual...'].str.replace(",", ".") # str is just for 1 column

# Change to the columns to numeric
# pd.to_numeric(df_data['Annual...']) # just 1 column
df_data[list_cols_tochange] = df_data[list_cols_tochange].apply(pd.to_numeric) # to several columns together
#df_data[list_cols_tochange] = df_data[list_cols_tochange].astype(float) # to several columns together # way #3

387,53


## Exercise 2

In [ ]:
df_data_ex2 = pd.read_csv("data_Top_50_US_Tech_Companies_2022-2023.csv", sep=";", header=None, names=my_columns[0], decimal=",")
#df_data_ex2.info()

# Filtering
print("Filter #1: " + str(len(df_data_ex2[df_data_ex2['Founding Year']>2000])))
print("Filter #2: " + str(len(df_data_ex2[(df_data_ex2['Founding Year']>2000) & (df_data_ex2['Annual Revenue 2022-2023 (USD in Billions)']>10)])))

Filter #1: 12
Filter #2: 4


## Exercise 3

In [ ]:
# Loading the data
df_excel = pd.read_excel("censusMadrid2022.xlsx", sheet_name="2022", header=4).dropna(subset=['Total'])
df_excel = df_excel.rename(columns={'Unnamed: 0': 'ID',
                                    'Unnamed: 1':'Name'})
df_excel.loc[1, 'ID'] = "0000" # Changing one value of record
print(df_excel[-25:] ) # Last 25 rows of the dataframe
# df_excel.tail(25) # way number 2

df_excel = df_excel[0:180] # Filtering
df_excel = df_excel.reset_index(drop=True) # Resetting the index (we can do it also at the beginning)

In [ ]:
sum1 = df_excel[1:]['Total'].sum()
value1 = df_excel.loc[df_excel['ID']=="0000"]['Total'].values[0]
if sum1 == value1:
  print("Sum is ok")
else:
  print("Sum is not ok")

Sum is ok


# Session 2

## Exercise 1

In [ ]:
df_wiki = pd.read_html("https://en.wikipedia.org/wiki/Spain_women's_national_football_team#Current_squad",
             match="María Isabel Rodríguez", flavor="html5lib")[0] # Selecting the first table (0) from the html that matches "María Isabel..."

In [ ]:
# SCRAPPING
df_wiki = df_wiki.dropna().reset_index(drop=True) # deleting the NaN and resetting index

In [ ]:
df_wiki[['Date of birth', 'age']] = df_wiki['Date of birth (age)'].str.split("\(age", expand=True)
#df_wiki['age'] = df_wiki['age'][0:3] # The other day this worked, but not today! Be careful with HTML

In [ ]:
df_wiki['age'] = df_wiki['age'].str.split("\)", expand=True)[0] # Splitting what is inside column 'age' by )
            #  and keeping only the string that is before it

In [ ]:
df_wiki['age'] = df_wiki['age'].str.strip() # Removing space

In [ ]:
df_wiki['age'] = df_wiki['age'].apply(pd.to_numeric) # Changing the type of the column to numeric
df_wiki.info() # Getting info from the DF

In [ ]:
df_wiki_grouped = df_wiki.groupby(by = ['Club']).agg({'age':['mean','min','max'], # [] as list of statistical functions to compute
                                    'Goals':'sum',
                                    'Player':'size'
                                    })

In [ ]:
df_wiki_grouped.sort_values(by= ('Goals', 'sum')) # multi-index

## Exercise 2

In [ ]:
df_wiki

In [ ]:
df_wiki['Date of birth'] = pd.to_datetime(df_wiki['Date of birth']) # Changing the type of data

In [ ]:
df_wiki.groupby(by=df_wiki['Date of birth'].dt.year).agg({'Goals':'sum'})

,Goals
Date of birth,
1990,55.0
1991,11.0
1992,29.0
1994,30.0
1996,39.0
1997,1.0
1998,14.0
1999,2.0
2000,19.0


## Exercise 3

In [ ]:
df_data_ex2 = pd.read_table("data_Top_50_US_Tech_Companies_2022-2023.csv", sep=";", header=None, decimal=",")

# Session 3

## Exercise 1

In [ ]:
from sqlite3 import connect
connection = connect(':memory:')
conn_cursor = connection.cursor()
#my_query_example = '''SELECT * FROM MARKET'''

In [ ]:
# QUERY TO CREATE A NEW TABLE
my_query_creation_table = '''create table if not exists MARKET (user_id integer, user_name text, product text, amount integer)'''
connection.execute(my_query_creation_table)
# be careful with if not exists!!!

# QUERY TO INSERT DATA IN THE TABLE
my_query_insert_data = '''insert into MARKET values
          (001, 'Monica', 'Eggs', 10),
          (002, 'Salva', 'Bread', 5),
          (003, 'Alessandra', 'Eggs', 8),
          (001, 'Monica', 'Tomatoes', 2)
        '''
connection.execute(my_query_insert_data)

In [ ]:
my_table_pandas = pd.read_sql("SELECT * FROM MARKET", connection)

In [ ]:
connection.close() # IMPORTANT!!!

In [ ]:
my_table_pandas

,user_id,user_name,product,amount
0,1,Monica,Eggs,10
1,2,Salva,Bread,5
2,3,Alessandra,Eggs,8
3,1,Monica,Tomatoes,2


In [ ]:
my_pivot = my_table_pandas.pivot(index="user_id", columns="product", values="amount")

In [ ]:
my_pivot#.info()

product,Bread,Eggs,Tomatoes
user_id,,,
1,NaN,10.0,2.0
2,5.0,NaN,NaN
3,NaN,8.0,NaN


## Exercise 2

In [ ]:
# OPTION 1
my_base_good = pd.read_json("good.json")['audio_features']
my_json_good = pd.json_normalize(my_base_good) # SIMPLE!!

'''
# OPTION 2
my_base_option2 = pd.read_json("good.json", typ="series")[0] # by default typ is frame but in our case it's not working with frame
my_json_option2 = pd.DataFrame.from_dict(my_base_option2)
'''

my_base_disl = pd.read_json("dislike.json")['audio_features']
my_json_disl = pd.json_normalize(my_base_disl) # SIMPLE!!

In [ ]:
len(pd.merge(my_json_good, my_json_disl, how="inner", on="id")) # 0 songs repeated

0

In [ ]:
my_json_good['like'] = "yes" # COL yes in good dataframe
my_json_disl['like'] = "no" # COL no in dislike dataframe
df_concat = pd.concat([my_json_good, my_json_disl], axis=0) # Concat of both dataframes

In [ ]:
df_concat.shape

(195, 19)

In [ ]:
print("SHAPE OF GOOD: " + str(my_json_good.shape) + " / SHAPE OF DISLIKE: " + str(my_json_disl.shape))

SHAPE OF GOOD: (100, 19) / SHAPE OF DISLIKE: (95, 19)


# Session 4

## Exercise 1 / Exercise 2

In [ ]:
my_json_good.to_csv("output_spotify.csv", index=False, float_format="%.3f") # Exercise 1
my_json_good.to_csv("output_spotify.txt", index=False, float_format="%.3f") # Exercise 2

In [ ]:
my_json_disl.to_csv("output_spotify.txt", index=False, float_format="%.3f", mode="a") # Exercise 1
my_json_disl.to_csv("output_spotify.txt", index=False, float_format="%.3f", mode="a") # Exercise 2

## Exercise 3

In [ ]:
my_json_good.to_excel("output_spotify.xlsx", index=False, float_format="%.3f", sheet_name="good")

In [ ]:
my_json_disl.to_excel("output_spotify.xlsx", index=False, float_format="%.3f", sheet_name="dislike")

In [ ]:
with pd.ExcelWriter("output_spotify.xlsx") as my_writer:
  my_json_good.to_excel(my_writer, index=False, float_format="%.3f", sheet_name="good")
  my_json_disl.to_excel(my_writer, index=False, float_format="%.3f", sheet_name="dislike")
  # using with is a "Good practice"!! We don't need to close the writer

In [ ]:
# my_json_good.loc[0:0,:] # option 1
# my_json_good.head(1) # option 2
my_row = pd.DataFrame(my_json_good.loc[0]).T # option 3 (transpose)

In [ ]:
with pd.ExcelWriter("output_spotify.xlsx", mode="a", if_sheet_exists="overlay") as my_writer2: # 'a' for appending not 'w'
  my_row.to_excel(my_writer2, sheet_name="dislike", header=False, index=False,
                  float_format="%.3f", startrow=len(my_json_disl)+1) # Adding the row at the end of the rows of the existing sheet